In [2]:
import pandas as pd
from pytrends.request import TrendReq
import time
import json

# Load the JSON file
with open('C:\PF\Projects\Recursion_6.0_JBBR\output.json', 'r') as file:
    uptownfunk_data = json.load(file)

# Extract unique words from the JSON content
keywords = list(set(word.lower() for entry in uptownfunk_data if 'text' in entry for word in entry['text'].split()))
regions = [
    "IN" 
]
timeframe = "now 7-d"  

<>:7: SyntaxWarning: invalid escape sequence '\P'
<>:7: SyntaxWarning: invalid escape sequence '\P'
C:\Users\harsh\AppData\Local\Temp\ipykernel_21200\3401258141.py:7: SyntaxWarning: invalid escape sequence '\P'
  with open('C:\PF\Projects\Recursion_6.0_JBBR\output.json', 'r') as file:


In [3]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
custom_stop_words = {'ago', 'am', 'i', 'you', 'me', 'your', 'it', 'for', 'to', 'of', 'in', 'on', 'with', 
                     'the', 'this', 'that', 'my', 'we', 'who', 'if', 'about', 'up', 'out', 'got', 
                     'get', 'what', 'a', 'also', 'from', 'and', 'or', 'but', 'so', 'just', 'at',
                     'be', 'is', 'are', 'was', 'were', 'been', 'do', 'does', 'did', 'will', 'would',
                     'should', 'can', 'could', 'may', 'might', 'must', 'shall', 'having', 'has', 'had',
                     'how', 'why', 'when', 'where', 'which', 'whom', 'whose', 'here', 'there', 'then',
                     'than', 'now', 'over', 'under', 'between', 'before', 'after', 'while', 'during',
                     'against', 'through', 'though', 'since', 'because', 'until', 'unless', 'nor', 'not',
                     'no', 'yes', 'yeah', 'yep', 'yup', 'ok', 'okay', 'sure', 'well', 'good', 'great',
                     'nice', 'awesome', 'cool', 'bad', 'terrible', 'horrible', 'awful', 'wonderful', 'fantastic',}
stop_words.update(custom_stop_words)

def clean_keyword(keyword):
    keyword = re.sub(r'[^\w\s]', '', keyword)
    keyword = keyword.lower()
    if keyword in stop_words or len(keyword) < 2:
        return None
    return keyword

keywords = {clean_keyword(kw) for kw in keywords if clean_keyword(kw)}

In [4]:
keywords

{'freeze',
 'dining',
 'personal',
 'cause',
 'multiply',
 'room',
 'pure',
 'bathroom',
 'forces',
 'evening',
 'vacation',
 'hydrogen',
 'easy',
 'every',
 'beyond',
 'production',
 'week',
 'disney',
 'awkward',
 'windows',
 'floor',
 'air',
 'ja',
 'parents',
 'vial',
 'makes',
 'pasadena',
 'stiffed',
 'felt',
 'mad',
 'shirt',
 'buffet',
 'make',
 'fills',
 'released',
 'promise',
 'sales',
 'toe',
 'todays',
 'amy',
 'cheese',
 'specific',
 'ordinary',
 'thinks',
 'rised',
 'enterprising',
 '18',
 'mind',
 'hold',
 'parking',
 'smelling',
 'peoples',
 'pacino',
 'sensors',
 'pumpkin',
 'reserved',
 'miss',
 'minds',
 'met',
 '16',
 'reveal',
 'assuming',
 'ask',
 'home',
 'publish',
 'noticed',
 'find',
 'bidder',
 'many',
 'young',
 'attention',
 'wagon',
 'painfully',
 'chevy',
 'explanation',
 'satisfying',
 'hair',
 'auditioned',
 'havent',
 'unethical',
 'law',
 'humidifier',
 'kind',
 'social',
 'id',
 'book',
 'go',
 'attempt',
 'west',
 'today',
 'said',
 'presents',
 'd

In [5]:
def chunk_keywords(keywords, chunk_size):
    """
    Splits a list of keywords into chunks of a specified size.
    
    Args:
        keywords (list): List of keywords to split.
        chunk_size (int): Size of each chunk.
    
    Returns:
        list: List of keyword chunks.
    """
    for i in range(0, len(keywords), chunk_size):
        yield keywords[i:i + chunk_size]

In [6]:
def analyze_search_trends(keywords, regions, timeframe):
    """
    Analyzes search trends for given keywords and regions with delays to avoid rate limits.
    
    Args:
        keywords (list): List of keywords to analyze (up to 5 per request).
        regions (list): List of region codes (e.g., 'US', 'UK').
        timeframe (str): Timeframe for trends (e.g., 'today 5-y').
    
    Returns:
        dict: Trend data for each region.
    """
    pytrends = TrendReq(hl='en-US', tz=360)  # Initialize pytrends
    all_data = {}

    # Split keywords into chunks of 5
    keyword_chunks = chunk_keywords(keywords, chunk_size=5)

    flag = 0

    for region in regions:
        for chunk in keyword_chunks:
            try:
                flag += 1
                # Build payload and fetch data
                pytrends.build_payload(chunk, cat=0, timeframe=timeframe, geo=region)
                data = pytrends.interest_over_time()
                if not data.empty:
                    if region not in all_data:
                        all_data[region] = data
                    else:
                        all_data[region] = all_data[region].join(data, how='outer', rsuffix='_dup')
                
                # Pause for 5 seconds to respect rate limits
                time.sleep(5)
            
            except Exception as e:
                flag += 1

                print(f"Error fetching data for {region} with keywords {chunk}: {e}")
                # Wait longer if an error occurs, then continue
                time.sleep(10)

            finally:
                if flag == 3:
                    break
    
    return all_data


In [7]:
# Convert keywords set to a list for chunking
keywords_list = list(keywords)

# Execute analysis
trend_data = analyze_search_trends(keywords_list, regions, timeframe)


# Display results
for region, data in trend_data.items():
    print(f"Trends for {region}:")
    print(data.head())
    print("\n")


c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\PF\Projects\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future

Trends for IN:
                     freeze  dining  personal  cause  multiply  isPartial  \
date                                                                        
2025-03-14 17:00:00       6      10        42     76         3      False   
2025-03-14 18:00:00       6      11        40     81         3      False   
2025-03-14 19:00:00       8      10        44     83         2      False   
2025-03-14 20:00:00       9       8        45     96         3      False   
2025-03-14 21:00:00       8      10        39     94         3      False   

                     room  pure  bathroom  forces  evening  isPartial_dup  \
date                                                                        
2025-03-14 17:00:00    79    25        29       3        6          False   
2025-03-14 18:00:00    90    27        33       3        5          False   
2025-03-14 19:00:00    91    28        37       4        5          False   
2025-03-14 20:00:00   100    26        41       4        5  

In [8]:
data

,freeze,dining,personal,cause,multiply,isPartial,room,pure,bathroom,forces,evening,isPartial_dup,vacation,hydrogen,easy,every,beyond,isPartial_dup
date,,,,,,,,,,,,,,,,,,
2025-03-14 17:00:00,6,10,42,76,3,False,79,25,29,3,6,False,9,9,45,29,5,False
2025-03-14 18:00:00,6,11,40,81,3,False,90,27,33,3,5,False,10,7,39,31,6,False
2025-03-14 19:00:00,8,10,44,83,2,False,91,28,37,4,5,False,10,8,33,32,7,False
2025-03-14 20:00:00,9,8,45,96,3,False,100,26,41,4,5,False,9,6,33,31,7,False
2025-03-14 21:00:00,8,10,39,94,3,False,91,24,36,2,4,False,8,8,35,31,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-21 13:00:00,8,12,52,72,6,False,56,23,15,3,16,False,7,12,85,31,7,False
2025-03-21 14:00:00,7,12,49,74,5,False,59,25,15,4,13,False,8,13,95,35,6,False
2025-03-21 15:00:00,7,12,51,70,5,False,58,25,16,3,9,False,7,13,84,33,6,False


In [ ]:
The scores in the trends data represent the relative search interest for the given keywords over time, as provided by Google Trends. Here's what they mean:

1. **Score Values**: Each score is a normalized value between 0 and 100, where:
    - `100` represents the peak popularity of the keyword during the specified timeframe.
    - `0` means there was insufficient data for that keyword at that time.

2. **Columns**: Each column corresponds to a keyword (e.g., `problem`, `poison`, etc.), and the values indicate the search interest for that keyword at the given timestamp.

3. **`isPartial` Columns**: These boolean columns indicate whether the data for that time period is incomplete or partial.

In summary, these scores help you understand how popular each keyword was in the specified region (`IN`) and timeframe (`now 7-d`).

SyntaxError: unterminated string literal (detected at line 1) (135476000.py, line 1)